In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from numpy import pi as π
import firedrake
import plot
from firedrake import inner, grad, dx, ds
from petsc4py import PETSc
from slepc4py import SLEPc

# Let's solve an eigenproblem

In this post I'll look at how to solve the Dirichlet eigenproblem for the Laplace operator.
The weak form of the eigenproblem is to find a function $u$ and a real number $\lambda$ such that, for all test functions $v$,

$$\int_\Omega\nabla u\cdot\nabla v\hspace{2pt}dx = \lambda\int_\Omega uv\hspace{2pt}dx.$$

When discretized via the finite element method, this becomes the generalized eigenvalue problem

$$\mathbf{A}\mathbf{u} = \lambda\mathbf{M}\mathbf{u}$$

where $\mathbf{A}$ is the stiffness matrix, $\mathbf{M}$ is the mass matrix, and $\mathbf{u}$ is the vector of finite element expansion coefficients.

In [ ]:
mesh = firedrake.Mesh('domain.msh')

We'll use the same domain as before.
In this demo we'll have to go a level deeper and call PETSc directly.

### Using SLEPc

First, we'll set up a function space and create forms for the stiffness and mass matrices.

In [ ]:
Q = firedrake.FunctionSpace(mesh, family='CG', degree=1)
ϕ = firedrake.TestFunction(Q)
ψ = firedrake.TrialFunction(Q)
a = inner(grad(ϕ), grad(ψ)) * dx
m = ϕ * ψ * dx

print(type(a), type(m))

bc = firedrake.DirichletBC(Q, firedrake.Constant(0), 'on_boundary')

Now the weird part comes!
Before, we've used `firedrake.assemble` to compute a real number from a rank-0 form.
Internally this does the loop over all the triangles of the mesh.
We can also use it to take a rank-2 form and get a matrix out of it.

In [ ]:
A = firedrake.assemble(a, bcs=bc).M.handle
M = firedrake.assemble(m).M.handle

print(type(A))

Next we start setting a bunch of options for how we're going to solve the eigenproblem.

In [ ]:
num_eigenvalues = 200

opts = PETSc.Options()
opts.setValue('eps_gen_hermitian', None)
opts.setValue('eps_target_real', None)
opts.setValue('eps_smallest_real', None)
opts.setValue('st_type', 'sinvert')
opts.setValue('st_ksp_type', 'cg')
opts.setValue('st_pc-type', 'jacobi')
opts.setValue('eps_tol', 1e-8)

These options state that we:

* are solving a generalized Hermitian eigenproblem
* want only the smallest real eigenvalues
* will apply a spectral transformation (ST), in this case inverting the matrix
* will apply this transformation using
  - the conjugate gradient method
  - a jacobi preconditioner
* want a residual tolerance of $10^{-8}$

Getting the options right is essential!
Without them the algorithm will fail to converge.

The remaining code sets up the eigenproblem and gets a solution out of it.

In [ ]:
eigensolver = SLEPc.EPS().create(comm=firedrake.COMM_WORLD)
eigensolver.setDimensions(num_eigenvalues)
eigensolver.setOperators(A, M)
eigensolver.setFromOptions()

In [ ]:
eigensolver.solve()

In [ ]:
num_converged = eigensolver.getConverged()
print(num_converged)

In [ ]:
Vr, Vi = A.getVecs()
λ = eigensolver.getEigenpair(0, Vr, Vi)

When we retrieve a result, it's stored as a PETSc vector, but to plot it we want to use a firedrake function.
Here we create an empty firedrake function defined on the space $Q$ and manually fill its vector entries from the PETSc vector.

In [ ]:
ϕ = firedrake.Function(Q)
ϕ.vector()[:] = Vr

# Get the sign right
if ϕ.vector()[0] < 0:
    ϕ *= -1

In [ ]:
fig, axes = plt.subplots()
axes.set_aspect('equal')
contours = plot.tricontourf(ϕ, 40, cmap='magma', axes=axes)
fig.colorbar(contours)
fig.show()

### Try it yourself

Some fun things you can do with the eigenvalues and eigenfunctions:

* Check how well the eigenvalue growth agrees with Weyl's law.
* Check how well the zero contours agree with the Courant nodal domain theorem.

How do the following changes affect the eigenvalues and eigenfunctions?

* Add spatially varying conductivity.
* Use a domain with more or fewer holes.
* Use Neumann or Robin boundary conditions instead of Dirichlet.